[View in Colaboratory](https://colab.research.google.com/github/AVJdataminer/AVJdataminer.github.io/blob/master/ColdFront_onColab.ipynb)

In [9]:
import os
os.listdir()

['sample_data', '.config']

In [10]:
import pandas as pd
url="https://s3.amazonaws.com/drivendata/data/55/public/consumption_train.csv"
train=pd.read_csv(url, index_col='Unnamed: 0')
train.head()

,series_id,timestamp,consumption,temperature
0,103088,2014-12-24 00:00:00,101842.233424,NaN
1,103088,2014-12-24 01:00:00,105878.048906,NaN
2,103088,2014-12-24 02:00:00,91619.105008,NaN
3,103088,2014-12-24 03:00:00,94473.706203,NaN
4,103088,2014-12-24 04:00:00,96976.755526,NaN


In [0]:
import numpy as np
train['temperature']= train['temperature'].fillna(train['temperature'].rolling(720, min_periods=360).mean())

In [0]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
#sns.heatmap(train.isnull(), cbar=False)

In [0]:
train.dropna(inplace=True)

In [0]:
import datetime

def time_stamp(df):
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["year"] = df["timestamp"].dt.year
    df["month"] =  df["timestamp"].dt.month
    df["day"] = df["timestamp"].dt.day 
time_stamp(train)

In [0]:
train['hour']=train['timestamp'].dt.hour
train['wkday']=train['timestamp'].dt.weekday
train['hour']=train['timestamp'].dt.strftime('%H')
train['day_of_yr']=train['timestamp'].dt.strftime('%j')

In [16]:
#read in meta data
url='https://s3.amazonaws.com/drivendata/data/55/public/meta.csv'
meta=pd.read_csv(url)
meta.head()

,series_id,surface,base_temperature,monday_is_day_off,tuesday_is_day_off,wednesday_is_day_off,thursday_is_day_off,friday_is_day_off,saturday_is_day_off,sunday_is_day_off
0,100003,x-large,low,False,False,False,False,False,True,True
1,100004,x-large,low,False,False,False,False,False,True,True
2,100006,x-small,low,False,False,False,False,False,True,True
3,100008,x-small,low,False,False,False,False,False,True,True
4,100010,x-small,low,False,False,False,False,False,True,True


In [17]:
meta=meta.iloc[:,0:3]
dummies = pd.get_dummies(meta['surface']).rename(columns=lambda x: str(x))
meta = pd.concat([meta, dummies], axis=1)
dummies = pd.get_dummies(meta['base_temperature']).rename(columns=lambda x: 'base_temp_' + str(x))
meta = pd.DataFrame(pd.concat([meta, dummies], axis=1))
columns = ['surface', 'base_temperature']
meta.drop(columns, inplace=True, axis=1)
meta.head()

,series_id,large,medium,small,x-large,x-small,xx-large,xx-small,base_temp_high,base_temp_low
0,100003,0,0,0,1,0,0,0,0,1
1,100004,0,0,0,1,0,0,0,0,1
2,100006,0,0,0,0,1,0,0,0,1
3,100008,0,0,0,0,1,0,0,0,1
4,100010,0,0,0,0,1,0,0,0,1


In [0]:
#join meta data to training data set
full=pd.merge(train, meta, left_on='series_id', right_on='series_id')

In [19]:
"""Examining Correlations"""
pd.DataFrame(full.corr()["consumption"].sort_values(ascending=False))

,consumption
consumption,1.000000
large,0.277053
small,0.154346
xx-large,0.127718
temperature,0.086170
year,0.064448
series_id,0.050276
base_temp_low,0.030459
month,0.020468
day,-0.013733


In [20]:
#read in cold start data
url='https://s3.amazonaws.com/drivendata/data/55/public/cold_start_test.csv'
cold=pd.read_csv(url, index_col='Unnamed: 0')
cold.head()

,series_id,timestamp,consumption,temperature
0,102781,2013-02-27 00:00:00,15295.740389,17.00
1,102781,2013-02-27 01:00:00,15163.209562,18.25
2,102781,2013-02-27 02:00:00,15022.264079,18.00
3,102781,2013-02-27 03:00:00,15370.420458,17.00
4,102781,2013-02-27 04:00:00,15303.103213,16.90


In [0]:
test_full=pd.merge(cold, meta, left_on='series_id', right_on='series_id')

In [0]:
def time_stamp(df):
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["year"] = df["timestamp"].dt.year
    df["month"] =  df["timestamp"].dt.month
    df["day"] = df["timestamp"].dt.day 
    df['hour'] = df['timestamp'].dt.hour
    df['wkday'] = df['timestamp'].dt.weekday
    
time_stamp(test_full)

In [0]:
test_full['day_of_yr']=test_full['timestamp'].dt.strftime('%j')
test_full['hour']=test_full['timestamp'].dt.strftime('%H')

In [24]:
full.head()

,series_id,timestamp,consumption,temperature,year,month,day,hour,wkday,day_of_yr,large,medium,small,x-large,x-small,xx-large,xx-small,base_temp_high,base_temp_low
0,103088,2014-12-30 17:00:00,143709.717612,-4.675,2014,12,30,17,1,364,0,0,0,0,1,0,0,0,1
1,103088,2014-12-30 18:00:00,157424.927949,-5.000,2014,12,30,18,1,364,0,0,0,0,1,0,0,0,1
2,103088,2014-12-31 14:00:00,153370.363034,-4.400,2014,12,31,14,2,365,0,0,0,0,1,0,0,0,1
3,103088,2014-12-31 15:00:00,137602.089769,-4.650,2014,12,31,15,2,365,0,0,0,0,1,0,0,0,1
4,103088,2014-12-31 16:00:00,131771.016064,-5.100,2014,12,31,16,2,365,0,0,0,0,1,0,0,0,1


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler

In [26]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.11.0


In [27]:
test_full.head()

,series_id,timestamp,consumption,temperature,large,medium,small,x-large,x-small,xx-large,xx-small,base_temp_high,base_temp_low,year,month,day,hour,wkday,day_of_yr
0,102781,2013-02-27 00:00:00,15295.740389,17.00,1,0,0,0,0,0,0,0,1,2013,2,27,00,2,058
1,102781,2013-02-27 01:00:00,15163.209562,18.25,1,0,0,0,0,0,0,0,1,2013,2,27,01,2,058
2,102781,2013-02-27 02:00:00,15022.264079,18.00,1,0,0,0,0,0,0,0,1,2013,2,27,02,2,058
3,102781,2013-02-27 03:00:00,15370.420458,17.00,1,0,0,0,0,0,0,0,1,2013,2,27,03,2,058
4,102781,2013-02-27 04:00:00,15303.103213,16.90,1,0,0,0,0,0,0,0,1,2013,2,27,04,2,058


In [0]:
Y_train=full.iloc[:,3].values
X_train=full.drop(['consumption','timestamp'], axis=1).values

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [31]:
X_train

array([[ 1.2049618 , -2.23115608, -1.93607843, ..., -0.36572415,
        -0.15983246,  0.15983246],
       [ 1.2049618 , -2.26759742, -1.93607843, ..., -0.36572415,
        -0.15983246,  0.15983246],
       [ 1.2049618 , -2.20032109, -1.93607843, ..., -0.36572415,
        -0.15983246,  0.15983246],
       ...,
       [ 0.15283212, -1.45683559,  0.7566498 , ..., -0.36572415,
        -0.15983246,  0.15983246],
       [ 0.15283212, -1.45632908,  0.7566498 , ..., -0.36572415,
        -0.15983246,  0.15983246],
       [ 0.15283212, -1.45576525,  0.7566498 , ..., -0.36572415,
        -0.15983246,  0.15983246]])

In [32]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1152      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,377
Trainable params: 5,377
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 100

# Store training stats
history = model.fit(X_train, Y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])


....................................................................................................

In [0]:
import matplotlib.pyplot as plt


def plot_history(history):
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [1000$]')
  plt.plot(history.epoch, np.array(history.history['mean_absolute_error']),
           label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
           label = 'Val loss')
  plt.legend()
  plt.ylim([0, 5])

plot_history(history)

In [0]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(X_train, Y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[early_stop, PrintDot()])

plot_history(history)

In [0]:
Y_test=full_test.iloc[:,3].values
X_test=full_test.drop(['consumption','timestamp'], axis=1).values
[loss, mae] = model.evaluate(X_test, Y_test, verbose=0)

print("Testing set Mean Abs Error: ${:7.2f}".format(mae * 1000))

In [0]:
test_predictions = model.predict(X_test).flatten()

plt.scatter(Y_test, test_predictions)
plt.xlabel('True Values [1000$]')
plt.ylabel('Predictions [1000$]')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
_ = plt.plot([-100, 100], [-100, 100])